In [1]:
import logging

import shapely

from build_airspaces import ProviderToken, parse_geometry_definition_file
from country_borders import CountryBordersConfig, get_country_borders


log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


class BorderProvider:
    def __init__(self, gdf_country_borders):
        self._gdf_country_borders = gdf_country_borders

    def get_border(self, border_name) -> shapely.LineString:
        rows = self._gdf_country_borders[
            self._gdf_country_borders["country"] == border_name
        ]
        if len(rows) == 1:
            return rows.iloc[0].geometry
        raise KeyError(f'no border for country "{border_name}"')


gdf_country_borders = get_country_borders(
    CountryBordersConfig(
        swissboundaries_path="./data/copyrighted/swissBOUNDARIES3D_1_5_LV95_LN02.gpkg",
        eu_boundaries_path="./data/copyrighted/CNTR_RG_01M_2024_4326.gpkg",
    )
)
providers = {ProviderToken.BORDER_PROVIDER: BorderProvider(gdf_country_borders)}
gdf_airspaces = parse_geometry_definition_file("./data/airspaces.csv", providers)
gdf_restricted_areas = parse_geometry_definition_file(
    "./data/restricted_areas.csv", providers
)
gdf_danger_areas = parse_geometry_definition_file("./data/danger_areas.csv", providers)

m = gdf_airspaces.explore(color="blue")
gdf_restricted_areas.explore(m=m, color="orange")
gdf_danger_areas.explore(m=m, color="yellow")